# Import the libraries

In [1]:
import pandas as pd
import numpy as np

from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.metrics import r2_score

import random
import time
import warnings
warnings.filterwarnings('ignore')

# Load the dataset

In [2]:
X, y =load_diabetes(return_X_y=True)

In [3]:
X.shape, y.shape

((442, 10), (442,))

# Train Test Split

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((353, 10), (89, 10), (353,), (89,))

# Linear Regression (OLS)

In [6]:
lr = LinearRegression()

## Fit the model

In [7]:
lr.fit(X_train, y_train)

LinearRegression()

## Prediction on test data and R2 Score

In [8]:
y_pred = lr.predict(X_test)

In [9]:
print('R2 Score using Linear Regression (OLS) is: ', r2_score(y_test, y_pred))

R2 Score using Linear Regression (OLS) is:  0.4526027629719196


## Coefficients and Slope

In [10]:
lr.coef_

array([  37.90402135, -241.96436231,  542.42875852,  347.70384391,
       -931.48884588,  518.06227698,  163.41998299,  275.31790158,
        736.1988589 ,   48.67065743])

In [11]:
lr.intercept_

151.34560453985995

# Mini-Batch Gradient Descent

In [12]:
class MBGDRegressor():

    def __init__(self, batch_size, learning_rate=0.01, epochs=100):
        self.coef_ = None
        self.intercept_ = None
        self.batch_size = batch_size
        self.lr = learning_rate
        self.epochs = epochs
    
    def fit(self, X_train, y_train):
        
        # Set the intercept to 0 and coefficents to 1
        # No of coefficents = No of columns in X_train
        self.intercept_ = 0
        self.coef_ = np.ones(X_train.shape[1])
        
        for i in range(self.epochs):
            for j in range(int(X_train.shape[0] / self.batch_size)):

                # Generate a random number to select a batch from X_train
                idx = random.sample(range(X_train.shape[0]), self.batch_size)

                # Y-Hat for selected batch
                y_hat = self.intercept_ + np.dot(X_train[idx], self.coef_)

                # Update Intercept
                # Partial Derivative of loss function w.r.t intercept for the selected row
                # -2 / n * sigma (yi - yihat) -> -2 * np.mean(y - yhat) (matrix multiplication)
                intercept_derivative = -2 * np.mean(y_train[idx] - y_hat)
                self.intercept_ = self.intercept_ - (self.lr * intercept_derivative)
    
                # Update Coefficients
                # Partial Derivative(s) of loss function w.r.t coefficients for the selected row
                # -2 / n * sigma (yi - yihat) * (xi1) -> -2 * dot product of y_train - y_hat and X_train
                coefficients_derivative = -2 * np.dot((y_train[idx] - y_hat), X_train[idx])
                self.coef_ = self.coef_ - (self.lr * coefficients_derivative)
        
        print('Intercept: ', self.intercept_)
        print('Coefficients: ', self.coef_)

    def predict(self, X_test):
        return self.intercept_ + np.dot(X_test, self.coef_)

In [13]:
mbgd = MBGDRegressor(batch_size=int(X_train.shape[0] / 30), learning_rate=0.01, epochs=100)

## Fit the model

In [14]:
mbgd.fit(X_train, y_train)

Intercept:  152.3500413288799
Coefficients:  [  41.10678995 -174.3399749   474.77404562  317.48052527  -49.87771973
  -94.59100891 -215.34626397  150.33351255  352.2671791   129.79400871]


## Prediction on test data and R2 Score

In [15]:
y_pred = mbgd.predict(X_test)

In [16]:
print('R2 Score using Mini-Batch Gradient Descent Regressor is: ', r2_score(y_test, y_pred))

R2 Score using Mini-Batch Gradient Descent Regressor is:  0.46040957928094517


# Using Sklearn

In [17]:
sgd = SGDRegressor(learning_rate='constant', eta0=0.01)

## Fit the model

In [18]:
batch_size = 100

for i in range(100):
    idx = random.sample(range(X_train.shape[0]), batch_size)
    sgd.partial_fit(X_train[idx], y_train[idx])

## Prediction on test data and R2 Score

In [19]:
y_pred = sgd.predict(X_test)

In [20]:
print('R2 Score using Stochastic Gradient Descent partial fit Regressor is: ', r2_score(y_test, y_pred))

R2 Score using Stochastic Gradient Descent partial fit Regressor is:  0.3379164808141658


## Coefficients and Slope

In [21]:
sgd.coef_

array([ 41.49549904, -13.62944331, 172.85578769, 123.77120623,
        32.85743624,  18.44240657, -98.33038872,  99.16665956,
       146.85489488,  97.68412128])

In [22]:
sgd.intercept_

array([153.74504925])